In [ ]:
# 1. numpy와 pandas를 둘 다 호환되는 버전으로 맞춰줍니다.
!pip install "numpy<2.0" "pandas>=2.0,<2.2"

# 2. 런타임 재시작 (중요)
import os
os.kill(os.getpid(), 9)

In [1]:
import pandas as pd
import numpy
print('numpy version:', numpy.__version__)
print('pandas version:', pd.__version__)

numpy version: 1.26.4
pandas version: 2.1.4


In [3]:
from google.colab import files
uploaded = files.upload()

Saving 블루블루엠.csv to 블루블루엠.csv


In [9]:
import pandas as pd

# 파일 업로드 및 컬럼명 확인
df = pd.read_csv('/content/블루블루엠.csv', encoding='utf-8-sig')
print(df.columns)             # 1. 컬럼명 확인
df.columns = df.columns.str.strip()  # 2. 혹시 모를 공백 제거

# price_map 등은 동일

# 아래처럼 실제 컬럼명에 맞게 변경 (예: '구분' → '메뉴명' 등)
def update_price(row, old_year, new_year):
    menu = str(row['구분']).strip()    # '구분' 대신 실제 컬럼명
    if pd.isna(row[f'{old_year}년 승인금액']):
        return None
    old_amt = row[f'{old_year}년 승인금액']
    try:
        old_price = price_map[menu][old_year]
        new_price = price_map[menu][new_year]
        return round((old_amt / old_price) * new_price, 0)
    except KeyError:
        return None

df['2023년 승인금액(환산)'] = df.apply(lambda row: update_price(row, '2013', '2023'), axis=1)
df['2024년 승인금액(환산)'] = df.apply(lambda row: update_price(row, '2014', '2024'), axis=1)

df.to_csv(output_file, encoding='utf-8-sig', index=False)
print(f'변환된 파일이 {output_file}에 저장되었습니다!')

Index(['매입사', '고객ID', '카드유형', '거래일', '거래시간', '승인금액', '할부', '인원'], dtype='object')


KeyError: '구분'

In [11]:
import pandas as pd

input_file = '/content/블루블루엠.csv'
output_file = '/content/블루블루엠_업데이트.csv'

base_menu = '평일 런치'

price_map = {
    '평일 런치': {'2013': 11900, '2023': 24900, '2014': 12900, '2024': 25900},
    '평일 디너': {'2013': 14900, '2023': 29900, '2014': 14900, '2024': 29900},
    '주말/공휴일': {'2013': 14900, '2023': 31900, '2014': 15900, '2024': 33900},
}

df = pd.read_csv(input_file, encoding='utf-8-sig')

# '승인금액', '인원' 컬럼 숫자형 변환 (콤마/공백 제거)
df['승인금액'] = df['승인금액'].astype(str).str.replace(',', '').str.strip()
df['승인금액'] = pd.to_numeric(df['승인금액'], errors='coerce')

df['인원'] = df['인원'].astype(str).str.replace(',', '').str.strip()
df['인원'] = pd.to_numeric(df['인원'], errors='coerce')

def 환산(row, old_year, new_year):
    old_amt = row['승인금액']
    if pd.isna(old_amt) or pd.isna(row['인원']) or row['인원'] == 0:
        return None
    old_price = price_map[base_menu][old_year]
    new_price = price_map[base_menu][new_year]
    return round((old_amt / old_price) * new_price, 0)

df['2023년 승인금액(환산)'] = df.apply(lambda row: 환산(row, '2013', '2023'), axis=1)
df['2024년 승인금액(환산)'] = df.apply(lambda row: 환산(row, '2014', '2024'), axis=1)

df.to_csv(output_file, encoding='utf-8-sig', index=False)
print(f'변환된 파일이 {output_file}에 저장되었습니다!')

변환된 파일이 /content/블루블루엠_업데이트.csv에 저장되었습니다!
